In [19]:
import pandas as pd 
from sklearn import tree
import numpy as np
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/GraphViz'

In [38]:
def read_clean(file_name):
    df = pd.read_csv(file_name)
    df["SibSp"] = df["SibSp"] + df["Parch"]
    df = df.replace({"Sex": {'male': 1, 'female': 0}, "Embarked": {"C": 10, "Q": 20, "S": 30}})

    del df["Name"]
    del df["Parch"]
    del df["Ticket"]
    del df["PassengerId"]
    del df["Cabin"]

    df.fillna(df.mean().astype(int), inplace=True)

    return df


In [39]:
train = read_clean("titanic/input/train.csv")
train_y = train.pop("Survived")

test = read_clean("titanic/input/test.csv")

answer = pd.read_csv("titanic/input/gender_submission.csv")
del answer['PassengerId']

In [40]:
def create_train(train, y, criterion, max_depth, test):
    clf = tree.DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
    clf.fit(train, y)
    if max_depth:
        file_name = "titanic/output/" + criterion + str(max_depth)
    else:
        file_name = "titanic/output/" + criterion
    dot_data = tree.export_graphviz(clf, out_file=None, 
                      feature_names=train.columns,
                      class_names=["Didn't", "Survived"],
                      filled=True, rounded=True,  
                      special_characters=True) 
    graph = graphviz.Source(dot_data)
    graph.render(file_name)
    result = clf.predict(test)
    out_df = pd.DataFrame(data=result, columns=["Survived"])
    out_df.to_csv(file_name + ".csv")
    return out_df

In [41]:
d1 = create_train(train, train_y, "gini", 5, test)
d2 = create_train(train, train_y, "gini", 4, test)
d3 = create_train(train, train_y, "entropy", 5, test)
d4 = create_train(train, train_y, "entropy", 4, test)

In [36]:
def accuracy(df, answer):
    acc = (df == answer).value_counts()[True]
    acc = int(acc) / len(d1)
    return acc

In [37]:
print("Accuracy for gini - 5 depth is %s" % accuracy(d1, answer))
print("Accuracy for gini - 4 depth is %s" % accuracy(d2, answer))
print("Accuracy for entropy - 5 depth is %s" % accuracy(d3, answer))
print("Accuracy for entropy - 4 depth is %s" % accuracy(d4, answer))

Accuracy for gini - 5 depth is 0.937799043062201
Accuracy for gini - 4 depth is 0.9617224880382775
Accuracy for entropy - 5 depth is 0.9497607655502392
Accuracy for entropy - 4 depth is 0.9497607655502392


In [42]:
test = pd.read_csv("titanic/input/test.csv")
del test["Name"]
del test["Parch"]
del test["Ticket"]
del test["Embarked"]
del test["Cabin"]
del test["Fare"]
del test["Pclass"]
del test["Sex"]
del test["Age"]
del test["SibSp"]
d2, test

a = test.join(d2)
a.to_csv("kaggle-nofare.csv", index=False)